# From chatgpt.com

## 1
Podrías hacer un código python que lance un query SQL a una base de datos SQL Server y que el resultado de dicha query la guarde como un archivo csv separado por ';' codificado utf-8 con BOM

Claro, aquí tienes un ejemplo de código Python que se conecta a una base de datos SQL Server, ejecuta una consulta SQL, y guarda el resultado en un archivo CSV con separador `;` y codificación `utf-8-sig` (que incluye el BOM):

### ✅ Requisitos previos

Necesitas instalar el paquete `pyodbc` si no lo tienes:

```bash
pip install pyodbc
```

---

### 💻 Código Python

```python
import pyodbc
import csv

# Parámetros de conexión (ajusta según tu configuración)
server = 'NOMBRE_DEL_SERVIDOR'
database = 'NOMBRE_DE_LA_BASE_DE_DATOS'
username = 'USUARIO'
password = 'CONTRASEÑA'

# Query SQL que quieres ejecutar
sql_query = 'SELECT * FROM tu_tabla'

# Nombre del archivo CSV de salida
output_file = 'resultado.csv'

# Establece la conexión
conn_str = (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    f'SERVER={server};DATABASE={database};UID={username};PWD={password}'
)

try:
    with pyodbc.connect(conn_str) as conn:
        cursor = conn.cursor()
        cursor.execute(sql_query)

        # Obtener nombres de columnas
        columns = [column[0] for column in cursor.description]

        # Escribir a CSV con separador ';' y codificación UTF-8 con BOM
        with open(output_file, mode='w', newline='', encoding='utf-8-sig') as f:
            writer = csv.writer(f, delimiter=';')
            writer.writerow(columns)  # Escribe encabezados
            for row in cursor:
                writer.writerow(row)

    print(f"Archivo CSV guardado exitosamente como '{output_file}'.")

except Exception as e:
    print("Ocurrió un error:", e)
```

---

¿Te gustaría que te ayudara a generar una versión parametrizable desde línea de comandos o con una interfaz simple?
- Afirmativo

Perfecto. A continuación, te presento una **versión mejorada y parametrizable desde línea de comandos**. Esta versión permite pasar los parámetros como argumentos, ideal para automatización o uso repetido.

---

### 💻 Código Python (parametrizable vía CLI)

Guarda esto como `export_sql_to_csv.py`:

```python
import argparse
import pyodbc
import csv
import sys

def export_query_to_csv(server, database, username, password, query, output_file):
    conn_str = (
        'DRIVER={ODBC Driver 17 for SQL Server};'
        f'SERVER={server};DATABASE={database};UID={username};PWD={password}'
    )

    try:
        with pyodbc.connect(conn_str) as conn:
            cursor = conn.cursor()
            cursor.execute(query)

            # Obtener nombres de columnas
            columns = [column[0] for column in cursor.description]

            # Escribir resultados en CSV
            with open(output_file, mode='w', newline='', encoding='utf-8-sig') as f:
                writer = csv.writer(f, delimiter=';')
                writer.writerow(columns)
                for row in cursor:
                    writer.writerow(row)

        print(f"✅ Archivo CSV guardado como '{output_file}'.")

    except Exception as e:
        print(f"❌ Error: {e}")
        sys.exit(1)

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Exporta el resultado de una consulta SQL Server a un archivo CSV.')

    parser.add_argument('--server', required=True, help='Nombre del servidor SQL Server')
    parser.add_argument('--database', required=True, help='Nombre de la base de datos')
    parser.add_argument('--username', required=True, help='Usuario de SQL Server')
    parser.add_argument('--password', required=True, help='Contraseña del usuario')
    parser.add_argument('--query', required=True, help='Consulta SQL que se va a ejecutar (en una sola línea o entre comillas)')
    parser.add_argument('--output', required=True, help='Ruta del archivo CSV de salida')

    args = parser.parse_args()

    export_query_to_csv(
        server=args.server,
        database=args.database,
        username=args.username,
        password=args.password,
        query=args.query,
        output_file=args.output
    )
```

---

### 🧪 Ejemplo de uso desde la terminal

```bash
python export_sql_to_csv.py \
  --server localhost \
  --database MiBase \
  --username sa \
  --password MiContraseña123 \
  --query "SELECT * FROM empleados" \
  --output empleados.csv
```

---

¿Quieres que también soporte autenticación de Windows (integrada)?


## 1' Prueba 1
- NO parametrizable vía CLI

In [1]:
## Libraries
import pyodbc
import csv

In [2]:
## Prueba 0- conexion y queries

# Parámetros de conexión (ajusta según tu configuración)
server = '172.31.119.50'
database = 'Bantotal'
username = 'usr_motor'
password = 'MTM.m0t0R2024'
table = ''

# Query SQL que quieres ejecutar
sql_query = 'select top 5 * from FSD010'
# sql_query = "select Pgcod,Aomod,Aosuc,Aomda,Aopap,Aocta,Aooper,Aosbop,Aotope,convert(datetime,Aofval,121) as Aofval,convert(datetime,Aofvto,121) \
#     as Aofvto,convert(float,Aopzo) as Aopzo,Aottas,convert(float,Aotasa) as Aotasa,Aotmor,convert(float,Aottac) as Aottac,Aotasc,Aotdia,Aotvto,Aotano,\
#         Aotint,Aodrev,convert(float,Aoimp) as Aoimp,Aopre,Aopre1,Aotcbi,convert(float,Aotcbi1) as Aotcbi1,Aoarb,Aoarb1,Aomd,Aomd1,Aonume,Aofnum,Aoafiv,\
#             Aocbcu,Aostat,Aoavis,Aoplus,Aoeven,convert(datetime,Aofe99,121) as Aofe99,Aocltcod,Aoperiod from fsd010 where aomod in (101,102,103,104) and \
#                 aopzo>0 and aofval <='20241231' union select Pgcod,Aomod,Aosuc,Aomda,Aopap,Aocta,Aooper,Aosbop,Aotope,convert(datetime,Aofval,121) as \
#                     Aofval,convert(datetime,Aofvto,121) as Aofvto,convert(float,Aopzo) as Aopzo,Aottas,convert(float,Aotasa) as Aotasa,Aotmor,\
#                         convert(float,Aottac) as Aottac,Aotasc,Aotdia,Aotvto,Aotano,Aotint,Aodrev,convert(float,Aoimp) as Aoimp,Aopre,Aopre1,Aotcbi,\
#                             convert(float,Aotcbi1) as Aotcbi1,Aoarb,Aoarb1,Aomd,Aomd1,Aonume,Aofnum,Aoafiv,Aocbcu,Aostat,Aoavis,Aoplus,Aoeven,\
#                                 convert(datetime,Aofe99,121) as Aofe99,Aocltcod,Aoperiod from fsd010 where aomod in (71) and aofval <='20241231'"

# Nombre del archivo CSV de salida
output_file = 'resultado.csv'

# Establece la conexión
conn_str = (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    f'SERVER={server};DATABASE={database};UID={username};PWD={password}'
)

try:
    with pyodbc.connect(conn_str) as conn:
        cursor = conn.cursor()
        cursor.execute(sql_query)

        # Obtener nombres de columnas
        columns = [column[0] for column in cursor.description]

        # Escribir a CSV con separador ';' y codificación UTF-8 con BOM
        with open(output_file, mode='w', newline='', encoding='utf-8-sig') as f:
            writer = csv.writer(f, delimiter=';')
            writer.writerow(columns)  # Escribe encabezados
            for row in cursor:
                writer.writerow(row)

    print(f"Archivo CSV guardado exitosamente como '{output_file}'.")

except Exception as e:
    print("Ocurrió un error:", e)

Archivo CSV guardado exitosamente como 'resultado.csv'.


In [3]:
## Prueba 1- tratar de leer fila a fila - y encabezado

# Parámetros de conexión (ajusta según tu configuración)
server = '172.31.119.50'
database = 'Bantotal'
username = 'usr_motor'
password = 'MTM.m0t0R2024'
table = ''

# Query SQL que quieres ejecutar
sql_query = 'select top 5 * from FSD010'
# sql_query = "select Pgcod,Aomod,Aosuc,Aomda,Aopap,Aocta,Aooper,Aosbop,Aotope,convert(datetime,Aofval,121) as Aofval,convert(datetime,Aofvto,121) \
#     as Aofvto,convert(float,Aopzo) as Aopzo,Aottas,convert(float,Aotasa) as Aotasa,Aotmor,convert(float,Aottac) as Aottac,Aotasc,Aotdia,Aotvto,Aotano,\
#         Aotint,Aodrev,convert(float,Aoimp) as Aoimp,Aopre,Aopre1,Aotcbi,convert(float,Aotcbi1) as Aotcbi1,Aoarb,Aoarb1,Aomd,Aomd1,Aonume,Aofnum,Aoafiv,\
#             Aocbcu,Aostat,Aoavis,Aoplus,Aoeven,convert(datetime,Aofe99,121) as Aofe99,Aocltcod,Aoperiod from fsd010 where aomod in (101,102,103,104) and \
#                 aopzo>0 and aofval <='20241231' union select Pgcod,Aomod,Aosuc,Aomda,Aopap,Aocta,Aooper,Aosbop,Aotope,convert(datetime,Aofval,121) as \
#                     Aofval,convert(datetime,Aofvto,121) as Aofvto,convert(float,Aopzo) as Aopzo,Aottas,convert(float,Aotasa) as Aotasa,Aotmor,\
#                         convert(float,Aottac) as Aottac,Aotasc,Aotdia,Aotvto,Aotano,Aotint,Aodrev,convert(float,Aoimp) as Aoimp,Aopre,Aopre1,Aotcbi,\
#                             convert(float,Aotcbi1) as Aotcbi1,Aoarb,Aoarb1,Aomd,Aomd1,Aonume,Aofnum,Aoafiv,Aocbcu,Aostat,Aoavis,Aoplus,Aoeven,\
#                                 convert(datetime,Aofe99,121) as Aofe99,Aocltcod,Aoperiod from fsd010 where aomod in (71) and aofval <='20241231'"

# Nombre del archivo CSV de salida
output_file = 'resultado.csv'

# Establece la conexión
conn_str = (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    f'SERVER={server};DATABASE={database};UID={username};PWD={password}'
)

try:
    with pyodbc.connect(conn_str) as conn:
        cursor = conn.cursor()
        cursor.execute(sql_query)
        print(cursor.description)
        for row in cursor:
            print(row)

    #     # Obtener nombres de columnas
    #     columns = [column[0] for column in cursor.description]

    #     # Escribir a CSV con separador ';' y codificación UTF-8 con BOM
    #     with open(output_file, mode='w', newline='', encoding='utf-8-sig') as f:
    #         writer = csv.writer(f, delimiter=';')
    #         writer.writerow(columns)  # Escribe encabezados
    #         for row in cursor:
    #             writer.writerow(row)

    # print(f"Archivo CSV guardado exitosamente como '{output_file}'.")

except Exception as e:
    print("Ocurrió un error:", e)

(('Pgcod', <class 'int'>, None, 5, 5, 0, False), ('Aomod', <class 'int'>, None, 5, 5, 0, False), ('Aosuc', <class 'int'>, None, 5, 5, 0, False), ('Aomda', <class 'int'>, None, 5, 5, 0, False), ('Aopap', <class 'int'>, None, 5, 5, 0, False), ('Aocta', <class 'int'>, None, 10, 10, 0, False), ('Aooper', <class 'int'>, None, 10, 10, 0, False), ('Aosbop', <class 'int'>, None, 5, 5, 0, False), ('Aotope', <class 'int'>, None, 5, 5, 0, False), ('Aofval', <class 'datetime.datetime'>, None, 23, 23, 3, True), ('Aofvto', <class 'datetime.datetime'>, None, 23, 23, 3, True), ('Aopzo', <class 'int'>, None, 10, 10, 0, True), ('Aottas', <class 'int'>, None, 5, 5, 0, True), ('Aotasa', <class 'decimal.Decimal'>, None, 10, 10, 6, True), ('Aotmor', <class 'decimal.Decimal'>, None, 10, 10, 6, True), ('Aottac', <class 'int'>, None, 5, 5, 0, True), ('Aotasc', <class 'decimal.Decimal'>, None, 10, 10, 6, True), ('Aotdia', <class 'int'>, None, 5, 5, 0, True), ('Aotvto', <class 'str'>, None, 1, 1, 0, True), ('Aot

## 2. 
Evitar notación científica
Evitar decimales en 0 (.0), si toda la columna es de enteros.

Para evitar que los números decimales aparezcan en **notación científica** o con **parte decimal innecesaria (.0)** en el archivo CSV, necesitas **convertir explícitamente** cada valor antes de escribirlo al archivo. Esto se puede hacer formateando los valores dentro del bucle de escritura.

Aquí tienes una versión modificada de tu código que:

1. Convierte números en formato decimal a cadena sin notación científica.
2. Quita `.0` si el valor decimal es un número entero.
3. Deja los demás valores (como cadenas o fechas) sin modificar.

### Código modificado:

```python
import pyodbc
import csv
import decimal

# Parámetros de conexión (ajusta según tu configuración)
server = 'NOMBRE_DEL_SERVIDOR'
database = 'NOMBRE_DE_LA_BASE_DE_DATOS'
username = 'USUARIO'
password = 'CONTRASEÑA'

# Query SQL que quieres ejecutar
sql_query = 'SELECT * FROM tu_tabla'

# Nombre del archivo CSV de salida
output_file = 'resultado.csv'

# Establece la conexión
conn_str = (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    f'SERVER={server};DATABASE={database};UID={username};PWD={password}'
)

def format_value(value):
    if isinstance(value, float) or isinstance(value, decimal.Decimal):
        if value == int(value):
            return str(int(value))  # Elimina parte decimal si es entero
        else:
            return format(value, 'f')  # No usar notación científica
    return str(value)  # Otros tipos (texto, fechas, etc.)

try:
    with pyodbc.connect(conn_str) as conn:
        cursor = conn.cursor()
        cursor.execute(sql_query)

        # Obtener nombres de columnas
        columns = [column[0] for column in cursor.description]

        # Escribir a CSV
        with open(output_file, mode='w', newline='', encoding='utf-8-sig') as f:
            writer = csv.writer(f, delimiter=';')
            writer.writerow(columns)
            for row in cursor:
                formatted_row = [format_value(value) for value in row]
                writer.writerow(formatted_row)

    print(f"Archivo CSV guardado exitosamente como '{output_file}'.")

except Exception as e:
    print("Ocurrió un error:", e)
```

### Resultado:

* `12.0000000000` → `12`
* `1.23E+5` → `123000`
* `45.67` → `45.67`

¿Quieres que te ayude a aplicar un formato numérico específico (como usar coma en vez de punto decimal)?
JM -> Lo pedí y queda mucho más abajo...

### 2.1 Probamos SOLO ENTEROS y SIN NOTACION CIENTIFICA.

In [4]:
## Functions
def format_value(value):
    if isinstance(value, float) or isinstance(value, decimal.Decimal):
        if value == int(value):
            return str(int(value))  # Elimina parte decimal si es entero
        else:
            return format(value, 'f')  # No usar notación científica
    return str(value)  # Otros tipos (texto, fechas, etc.)

# Parámetros de conexión (ajusta según tu configuración)
server = '172.31.119.50'
database = 'Bantotal'
username = 'usr_motor'
password = 'MTM.m0t0R2024'

# Query SQL que quieres ejecutar
sql_query = 'select top 5 * from FSD010'

# Establece la conexión
conn_str = (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    f'SERVER={server};DATABASE={database};UID={username};PWD={password}'
)

try:
    with pyodbc.connect(conn_str) as conn:
        cursor = conn.cursor()
        cursor.execute(sql_query)
        rows = cursor.fetchall()
        descript = cursor.description
        print(descript)
        # print(cursor.description)
        # for row in cursor:
        #     print(row)
        # print(row[0])
        # print(row[1])
        # print(cursor)

except Exception as e:
    print("Ocurrió un error:", e)

(('Pgcod', <class 'int'>, None, 5, 5, 0, False), ('Aomod', <class 'int'>, None, 5, 5, 0, False), ('Aosuc', <class 'int'>, None, 5, 5, 0, False), ('Aomda', <class 'int'>, None, 5, 5, 0, False), ('Aopap', <class 'int'>, None, 5, 5, 0, False), ('Aocta', <class 'int'>, None, 10, 10, 0, False), ('Aooper', <class 'int'>, None, 10, 10, 0, False), ('Aosbop', <class 'int'>, None, 5, 5, 0, False), ('Aotope', <class 'int'>, None, 5, 5, 0, False), ('Aofval', <class 'datetime.datetime'>, None, 23, 23, 3, True), ('Aofvto', <class 'datetime.datetime'>, None, 23, 23, 3, True), ('Aopzo', <class 'int'>, None, 10, 10, 0, True), ('Aottas', <class 'int'>, None, 5, 5, 0, True), ('Aotasa', <class 'decimal.Decimal'>, None, 10, 10, 6, True), ('Aotmor', <class 'decimal.Decimal'>, None, 10, 10, 6, True), ('Aottac', <class 'int'>, None, 5, 5, 0, True), ('Aotasc', <class 'decimal.Decimal'>, None, 10, 10, 6, True), ('Aotdia', <class 'int'>, None, 5, 5, 0, True), ('Aotvto', <class 'str'>, None, 1, 1, 0, True), ('Aot

Perfecto. Si quieres que los números decimales usen **coma (`,`) como separador decimal**, como es común en muchos países hispanohablantes, puedes modificar la función `format_value` para que reemplace el punto por una coma **solo en números**.

Aquí te muestro cómo hacerlo:

---

### ✅ Versión actualizada con coma como separador decimal:

```python
import pyodbc
import csv
import decimal

# Parámetros de conexión
server = 'NOMBRE_DEL_SERVIDOR'
database = 'NOMBRE_DE_LA_BASE_DE_DATOS'
username = 'USUARIO'
password = 'CONTRASEÑA'

# Query SQL
sql_query = 'SELECT * FROM tu_tabla'

# Archivo CSV de salida
output_file = 'resultado.csv'

# Cadena de conexión
conn_str = (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    f'SERVER={server};DATABASE={database};UID={username};PWD={password}'
)

# Formatear valores
def format_value(value):
    if isinstance(value, float) or isinstance(value, decimal.Decimal):
        if value == int(value):
            return str(int(value))
        else:
            return format(value, 'f').replace('.', ',')  # Reemplaza punto por coma
    return str(value)

try:
    with pyodbc.connect(conn_str) as conn:
        cursor = conn.cursor()
        cursor.execute(sql_query)

        # Nombres de columnas
        columns = [column[0] for column in cursor.description]

        # Escribir CSV
        with open(output_file, mode='w', newline='', encoding='utf-8-sig') as f:
            writer = csv.writer(f, delimiter=';')
            writer.writerow(columns)
            for row in cursor:
                formatted_row = [format_value(value) for value in row]
                writer.writerow(formatted_row)

    print(f"Archivo CSV guardado exitosamente como '{output_file}'.")

except Exception as e:
    print("Ocurrió un error:", e)
```

---

### 🔍 Ejemplo de conversión:

| Valor en base de datos | En CSV       |
| ---------------------- | ------------ |
| `123.0`                | `123`        |
| `123456.789`           | `123456,789` |
| `1.0E+4`               | `10000`      |
| `1.23`                 | `1,23`       |

---

¿Quieres que también se apliquen separadores de miles (por ejemplo, `1.234,56` en lugar de `1234,56`)?

## 3. cursor.description
+ algo sobre SQL server OFFSET - FETCH (TOP antes)

¡Perfecto! Para hacer que el número de fila sea dinámico, simplemente define una variable en Python que indique qué registro deseas obtener, y luego ajústalo en la consulta SQL.

### ✅ Ejemplo completo: obtener el **registro N** desde Python

```python
registro_n = 4  # Por ejemplo, obtener el cuarto registro (índice comienza en 1)
offset = registro_n - 1  # OFFSET comienza en 0

sql_query = f"""
SELECT *
FROM tu_tabla
ORDER BY id  -- reemplaza 'id' con la columna por la cual quieras ordenar
OFFSET {offset} ROWS FETCH NEXT 1 ROWS ONLY
"""

with pyodbc.connect(conn_str) as conn:
    cursor = conn.cursor()
    cursor.execute(sql_query)
    row = cursor.fetchone()
    if row:
        print(row)
    else:
        print(f"No se encontró el registro número {registro_n}.")
```

🔒 **Sugerencia de seguridad:** Para evitar SQL Injection, usa parámetros en lugar de interpolar con `f""` si el valor de `registro_n` proviene de una fuente externa.

¿Quieres ver cómo se hace usando parámetros en lugar de `f-string`?


¡Claro! `cursor.description` es una **tupla de tuplas**, donde cada sub-tupla describe una columna del resultado de tu consulta SQL.

Cada sub-tupla tiene generalmente 7 elementos, aunque **solo el primero (nombre de la columna)** es consistentemente usado en la mayoría de los drivers (incluido `pyodbc` con SQL Server). Los otros elementos dependen del driver y pueden venir como `None`.

### 📘 Estructura de cada elemento en `cursor.description`:

```python
(name, type_code, display_size, internal_size, precision, scale, null_ok)
```

Aquí te explico uno por uno:

| Índice | Elemento        | Significado                                                              |
| ------ | --------------- | ------------------------------------------------------------------------ |
| 0      | `name`          | **Nombre de la columna** (string)                                        |
| 1      | `type_code`     | Tipo de dato en el lenguaje (depende del driver, ej. int, str, etc.)     |
| 2      | `display_size`  | Tamaño máximo que puede ocupar al mostrar (normalmente `None` en pyodbc) |
| 3      | `internal_size` | Tamaño en bytes (por ejemplo, 4 para un `INT`, 8 para un `FLOAT`)        |
| 4      | `precision`     | Precisión para números decimales (número total de dígitos)               |
| 5      | `scale`         | Escala decimal (número de dígitos a la derecha del punto decimal)        |
| 6      | `null_ok`       | Booleano que indica si acepta `NULL`                                     |

### 🧪 Ejemplo práctico:

Supongamos que tu consulta devuelve una tabla con una columna llamada `edad` de tipo `INT`, y otra llamada `nombre` de tipo `VARCHAR(50)`:

```python
for col in cursor.description:
    print(col)
```

Podrías ver algo como:

```python
('edad', <class 'int'>, None, None, None, None, True)
('nombre', <class 'str'>, None, 50, None, None, True)
```

💡 En muchos casos, lo más útil es solo el primer elemento (el nombre de la columna), así que normalmente se hace:

```python
columnas = [desc[0] for desc in cursor.description]
```

¿Quieres ver un ejemplo con datos reales de tu consulta para inspeccionar `cursor.description` en acción?


Absolutely! Below is the **fully translated script in English**, with the following updates:

* All **text, comments, and output** are now in **English**.
* The **record number** is taken from the **first command-line argument**.
* It will print the selected record's metadata and values in a formatted table.
* It will export the output to both `.txt` and `.csv` files.

---

### ✅ Full Script in English (record number via CLI argument)

```python
import pyodbc
import csv
import sys

# Ensure the user provided a record number argument
if len(sys.argv) < 2:
    print("Usage: python script.py <record_number>")
    sys.exit(1)

try:
    record_number = int(sys.argv[1])
    if record_number <= 0:
        raise ValueError
except ValueError:
    print("The record number must be a positive integer.")
    sys.exit(1)

# Output filenames
txt_file = f"record_{record_number}.txt"
csv_file = f"record_{record_number}.csv"

# Replace these with your actual connection string and query
conn_str = "your_connection_string_here"
sql_query = "SELECT * FROM your_table_here"

with pyodbc.connect(conn_str) as conn:
    cursor = conn.cursor()
    cursor.execute(sql_query)

    rows = cursor.fetchall()
    if len(rows) < record_number:
        print(f"There are only {len(rows)} records available.")
        sys.exit(1)

    selected_row = rows[record_number - 1]
    column_info = cursor.description

    # Table headers
    headers = [
        "Field", "type_code", "display_size", "internal_size",
        "precision", "scale", "null_ok", "Value"
    ]

    # Prepare data for output
    table_rows = []
    for i, col in enumerate(column_info):
        row_data = [
            col[0],
            str(col[1]),
            col[2],
            col[3],
            col[4],
            col[5],
            col[6],
            selected_row[i]
        ]
        table_rows.append(row_data)

    # Print to console
    print(f"\nFull information for record #{record_number}:\n")
    print(f"{headers[0]:<20} {headers[1]:<30} {headers[2]:<13} {headers[3]:<14} "
          f"{headers[4]:<10} {headers[5]:<7} {headers[6]:<8} {headers[7]}")
    print("-" * 130)
    for row in table_rows:
        print(f"{str(row[0]):<20} {str(row[1]):<30} {str(row[2]):<13} {str(row[3]):<14} "
              f"{str(row[4]):<10} {str(row[5]):<7} {str(row[6]):<8} {row[7]}")

    # Save to .txt
    with open(txt_file, 'w', encoding='utf-8') as f:
        f.write(f"Full information for record #{record_number}:\n\n")
        f.write(f"{headers[0]:<20} {headers[1]:<30} {headers[2]:<13} {headers[3]:<14} "
                f"{headers[4]:<10} {headers[5]:<7} {headers[6]:<8} {headers[7]}\n")
        f.write("-" * 130 + "\n")
        for row in table_rows:
            f.write(f"{str(row[0]):<20} {str(row[1]):<30} {str(row[2]):<13} {str(row[3]):<14} "
                    f"{str(row[4]):<10} {str(row[5]):<7} {str(row[6]):<8} {row[7]}\n")

    # Save to .csv
    with open(csv_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(headers)
        writer.writerows(table_rows)

    print(f"\n✔️ Output exported to '{txt_file}' and '{csv_file}' successfully.")
```

---

### 🧪 How to run it from terminal:

```bash
python script.py 4
```

This will show and export information about the **4th record**.

Would you like me to add timestamp-based filenames as well?



### 3.1 Caso con análisis de registro
- Este script es muy útil para depuración o análisis detallado de un solo registro.
- Voy a crear reganalys.py (porque el nro de registro se hace por cli)

## 4 Argumentos en vs-code

¡Buena pregunta! Para ejecutar un script con argumentos en **Visual Studio Code (VS Code)**, tienes varias opciones. Aquí te explico las más comunes y efectivas:

---

### ✅ Opción 1: Ejecutar desde el **Terminal integrado**

1. Abre tu archivo Python (por ejemplo, `script.py`) en VS Code.
2. Presiona \`Ctrl + \`\` (tilde invertida) o ve a **View > Terminal** para abrir el terminal integrado.
3. Ejecuta el script como lo harías desde cualquier terminal:

   ```bash
   python script.py 4
   ```

   *(Esto ejecuta el script y pasa `4` como primer argumento.)*

---

### ✅ Opción 2: Configurar **launch.json** para ejecutar con argumentos

1. Ve al panel de **Run and Debug** (ícono de ▶️ en la barra lateral izquierda o presiona `Ctrl + Shift + D`).
2. Haz clic en el ícono de engranaje ⚙️ arriba a la derecha (esto abrirá el archivo `launch.json`).
3. Crea o edita una configuración como esta:

```json
{
    "version": "0.2.0",
    "configurations": [
        {
            "name": "Python: Run with arguments",
            "type": "python",
            "request": "launch",
            "program": "${file}",          // Ejecuta el archivo abierto
            "console": "integratedTerminal",
            "args": ["4"]                  // Aquí pones tus argumentos
        }
    ]
}
```

4. Guarda `launch.json`.
5. Luego simplemente presiona **F5** para ejecutar con el argumento.

---

### ✅ Opción 3: Pasar argumentos en un script de prueba

Si solo estás haciendo pruebas rápidas, puedes modificar el inicio del script para simular argumentos:

```python
import sys
sys.argv = ["script.py", "4"]  # Simula pasar '4' como argumento
```

Este truco es útil si estás probando sin querer modificar la configuración de VS Code.

---

¿Te gustaría que te ayude a configurar `launch.json` para tu proyecto actual?


Perfecto. Aquí tienes una guía paso a paso para configurar `launch.json` en **VS Code** para que tu script Python acepte argumentos como `4`, `10`, etc.

---

### 🛠️ Paso a paso para configurar `launch.json` en VS Code

#### 1. Abre VS Code y carga tu proyecto

Abre la carpeta o el archivo `.py` que contiene tu script, por ejemplo: `script.py`.

#### 2. Ve al panel **Run and Debug**

* Haz clic en el icono de ▶️ con el insecto (barra lateral izquierda), o presiona `Ctrl + Shift + D`.

#### 3. Haz clic en **"create a launch.json file"** si aún no lo tienes

* Si te lo pregunta, selecciona **Python** como tipo de entorno.

#### 4. Reemplaza o edita la configuración por esta:

```json
{
    "version": "0.2.0",
    "configurations": [
        {
            "name": "Run script with argument",
            "type": "python",
            "request": "launch",
            "program": "${file}",                // Ejecuta el archivo abierto
            "console": "integratedTerminal",     // Usa el terminal de VS Code
            "args": ["4"]                        // <- Aquí va el argumento que quieres pasar
        }
    ]
}
```

> 🔁 Cambia `"4"` por el número de registro que quieras probar.

#### 5. Guarda el archivo `launch.json`

---

### ▶️ Ejecutar

1. Asegúrate de tener abierto el archivo del script.
2. Presiona **F5** o haz clic en **Run** arriba.
3. El script se ejecutará y tomará `4` como el argumento de línea de comandos.

---

### 📝 Consejo útil

Puedes cambiar el valor dentro de `"args": ["4"]` cuantas veces quieras sin modificar tu código.

---

¿Quieres que el script tome **múltiples argumentos**, o solo uno es suficiente para tu caso?


¡Perfecto! Si deseas que tu script acepte **múltiples argumentos**, puedes hacerlo fácilmente usando `sys.argv` como lista de argumentos o, mejor aún, usando `argparse` para mayor claridad y control.

A continuación te muestro:

---

### ✅ 1. Cómo pasar múltiples argumentos desde `launch.json`

Supongamos que quieres pasar:

* `4` → el número de registro
* `"output_folder"` → el nombre de una carpeta

#### 🔧 En `launch.json`:

```json
"args": ["4", "output_folder"]
```

---

### ✅ 2. Cómo recibirlos en tu script

#### 🔄 Opción simple (usando `sys.argv`):

```python
import sys

# Skip the script name itself
if len(sys.argv) < 3:
    print("Usage: python script.py <record_number> <output_folder>")
    sys.exit(1)

record_number = int(sys.argv[1])
output_folder = sys.argv[2]

print(f"Record #: {record_number}")
print(f"Output folder: {output_folder}")
```

#### ✅ Opción profesional (usando `argparse`):

```python
import argparse

parser = argparse.ArgumentParser(description="Export a selected record with metadata.")
parser.add_argument("record_number", type=int, help="The number of the record to export")
parser.add_argument("output_folder", type=str, help="The folder where output files will be saved")

args = parser.parse_args()

print(f"Record #: {args.record_number}")
print(f"Output folder: {args.output_folder}")
```

Luego en `launch.json`, mantienes:

```json
"args": ["4", "my_folder"]
```

---

¿Te gustaría que adapte el script anterior completo para usar `argparse` y guardar los archivos en una carpeta que también se pasa por parámetro?


¡Perfecto! A continuación tienes el **script completo y profesional** que:

✅ Usa `argparse` para manejar argumentos desde la línea de comandos
✅ Recibe dos argumentos:

* `record_number` (registro a mostrar)
* `output_folder` (directorio donde guardar los archivos)
  ✅ Exporta el resultado a `.txt` y `.csv` dentro de esa carpeta
  ✅ Crea la carpeta si no existe
  ✅ Imprime el resultado como tabla legible en consola

---

### ✅ Script completo en inglés con `argparse` y carpeta de salida

```python
import pyodbc
import csv
import argparse
import os
from pathlib import Path

# --- Argument parsing ---
parser = argparse.ArgumentParser(description="Export detailed metadata and values for a selected record.")
parser.add_argument("record_number", type=int, help="Record number to export (1-based index)")
parser.add_argument("output_folder", type=str, help="Folder where output files will be saved")
args = parser.parse_args()

record_number = args.record_number
output_folder = args.output_folder

# Validate record number
if record_number <= 0:
    print("Record number must be a positive integer.")
    exit(1)

# --- Setup output paths ---
Path(output_folder).mkdir(parents=True, exist_ok=True)  # Create folder if it doesn't exist
txt_file = os.path.join(output_folder, f"record_{record_number}.txt")
csv_file = os.path.join(output_folder, f"record_{record_number}.csv")

# --- Connection and query (replace with yours) ---
conn_str = "your_connection_string_here"
sql_query = "SELECT * FROM your_table_here"

with pyodbc.connect(conn_str) as conn:
    cursor = conn.cursor()
    cursor.execute(sql_query)

    rows = cursor.fetchall()
    if len(rows) < record_number:
        print(f"There are only {len(rows)} records available.")
        exit(1)

    selected_row = rows[record_number - 1]
    column_info = cursor.description

    headers = [
        "Field", "type_code", "display_size", "internal_size",
        "precision", "scale", "null_ok", "Value"
    ]

    table_rows = []
    for i, col in enumerate(column_info):
        row_data = [
            col[0],                    # Field name
            str(col[1]),               # Python type
            col[2],                    # Display size
            col[3],                    # Internal size
            col[4],                    # Precision
            col[5],                    # Scale
            col[6],                    # Null allowed
            selected_row[i]            # Value
        ]
        table_rows.append(row_data)

    # --- Print to console ---
    print(f"\nFull metadata and values for record #{record_number}:\n")
    print(f"{headers[0]:<20} {headers[1]:<30} {headers[2]:<13} {headers[3]:<14} "
          f"{headers[4]:<10} {headers[5]:<7} {headers[6]:<8} {headers[7]}")
    print("-" * 130)
    for row in table_rows:
        print(f"{str(row[0]):<20} {str(row[1]):<30} {str(row[2]):<13} {str(row[3]):<14} "
              f"{str(row[4]):<10} {str(row[5]):<7} {str(row[6]):<8} {row[7]}")

    # --- Write to .txt ---
    with open(txt_file, 'w', encoding='utf-8') as f:
        f.write(f"Full metadata and values for record #{record_number}:\n\n")
        f.write(f"{headers[0]:<20} {headers[1]:<30} {headers[2]:<13} {headers[3]:<14} "
                f"{headers[4]:<10} {headers[5]:<7} {headers[6]:<8} {headers[7]}\n")
        f.write("-" * 130 + "\n")
        for row in table_rows:
            f.write(f"{str(row[0]):<20} {str(row[1]):<30} {str(row[2]):<13} {str(row[3]):<14} "
                    f"{str(row[4]):<10} {str(row[5]):<7} {str(row[6]):<8} {row[7]}\n")

    # --- Write to .csv ---
    with open(csv_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(headers)
        writer.writerows(table_rows)

    print(f"\n✔️ Output exported to:\n- {txt_file}\n- {csv_file}")
```

---

### ▶️ Example: Run from VS Code or terminal

```bash
python script.py 4 output_folder
```

This will extract the **4th record** and save the results in a folder named `output_folder`.

---

¿Quieres que los nombres de archivo incluyan también la fecha y hora para evitar sobrescrituras?


Excelente solicitud. A continuación te entrego una versión **modificada y mejorada** de tu script con todos los cambios que pediste:

---

### ✅ Cambios implementados:

1. ✅ **Primer argumento**: SQL query
2. ✅ **Segundo argumento**: Número de registro
3. ✅ Si **no se pasa ningún argumento**, el script muestra una tabla de ayuda con ejemplos
4. ✅ Los **archivos de salida** `.txt` y `.csv` llevan la **fecha y hora** en el nombre para evitar sobrescrituras
5. ✅ El contenido de la tabla de ayuda también se guarda como `.csv`

---

### 🆕 Código actualizado:

```python
## Libraries
import pyodbc
import csv
import sys
from datetime import datetime

## Constants
server = '172.31.119.50'
database = 'Bantotal'
username = 'usr_motor'
password = 'MTM.m0t0R2024'

conn_str = (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    f'SERVER={server};DATABASE={database};UID={username};PWD={password}'
)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

## No arguments: show help table and write to CSV
if len(sys.argv) < 3:
    print("\n🛈 Usage: python script.py <SQL_QUERY> <RECORD_NUMBER>\n")
    help_headers = ["Example usage", "Description"]
    help_data = [
        ["SELECT TOP 5 * FROM fsd010", "Get first 5 records from fsd010"],
        ["SELECT * FROM customers WHERE country = 'USA'", "All US customers"],
        ["SELECT * FROM table", "Full table export"],
    ]

    # Print help table
    print(f"{help_headers[0]:<50} {help_headers[1]}")
    print("-" * 80)
    for row in help_data:
        print(f"{row[0]:<50} {row[1]}")

    # Save help as CSV
    help_file = f"query_help_{timestamp}.csv"
    with open(help_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(help_headers)
        writer.writerows(help_data)

    print(f"\nℹ️ Help saved to '{help_file}'")
    sys.exit(0)

## Read input arguments
sql_query = sys.argv[1]
try:
    record_number = int(sys.argv[2])
    if record_number <= 0:
        raise ValueError
except ValueError:
    print("The record number must be a positive integer.")
    sys.exit(1)

## Output filenames with timestamp
txt_file = f"record_{record_number}_{timestamp}.txt"
csv_file = f"record_{record_number}_{timestamp}.csv"

## Main
with pyodbc.connect(conn_str) as conn:
    cursor = conn.cursor()
    cursor.execute(sql_query)

    rows = cursor.fetchall()
    if len(rows) < record_number:
        print(f"There are only {len(rows)} records available.")
        sys.exit(1)

    selected_row = rows[record_number - 1]
    column_info = cursor.description

    headers = [
        "Field_name", "type_code", "display_size", "internal_size",
        "precision", "scale", "null_ok", "Value"
    ]

    table_rows = []
    for i, col in enumerate(column_info):
        row_data = [
            col[0],
            str(col[1]),
            col[2],
            col[3],
            col[4],
            col[5],
            col[6],
            selected_row[i]
        ]
        table_rows.append(row_data)

    # Print to console
    print(f"\nFull information for record #{record_number}:\n")
    print(f"{headers[0]:<20} {headers[1]:<30} {headers[2]:<13} {headers[3]:<14} "
          f"{headers[4]:<10} {headers[5]:<7} {headers[6]:<8} {headers[7]}")
    print("-" * 130)
    for row in table_rows:
        print(f"{str(row[0]):<20} {str(row[1]):<30} {str(row[2]):<13} {str(row[3]):<14} "
              f"{str(row[4]):<10} {str(row[5]):<7} {str(row[6]):<8} {row[7]}")

    # Save to .txt
    with open(txt_file, 'w', encoding='utf-8') as f:
        f.write(f"Full information for record #{record_number}:\n\n")
        f.write(f"{headers[0]:<20} {headers[1]:<30} {headers[2]:<13} {headers[3]:<14} "
                f"{headers[4]:<10} {headers[5]:<7} {headers[6]:<8} {headers[7]}\n")
        f.write("-" * 130 + "\n")
        for row in table_rows:
            f.write(f"{str(row[0]):<20} {str(row[1]):<30} {str(row[2]):<13} {str(row[3]):<14} "
                    f"{str(row[4]):<10} {str(row[5]):<7} {str(row[6]):<8} {row[7]}\n")

    # Save to .csv
    with open(csv_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(headers)
        writer.writerows(table_rows)

    print(f"\n✔️ Output exported to:\n- {txt_file}\n- {csv_file}")
    input("\nPress 'Enter' to finish.")
```

---

### ▶️ Ejemplo de uso desde terminal o VS Code:

```bash
python script.py "SELECT TOP 5 * FROM fsd010" 3
```

Esto seleccionará el **3er registro** del resultado y exportará los archivos con timestamp.

---

¿Deseas que el script también valide que la consulta no esté vacía o que devuelva al menos un campo?


Perfecto. Aquí tienes una **versión final mejorada** del script que incluye:

---

### ✅ Cambios añadidos:

1. ✅ **Valida que la consulta no esté vacía**
2. ✅ **Verifica que el resultado tenga columnas** (`cursor.description` no sea `None`)
3. ✅ Mensajes de error claros si no se obtienen resultados o columnas
4. ✅ Mantiene la exportación y formato de salida con fecha y hora

---

### 🆕 Código final:

```python
## Libraries
import pyodbc
import csv
import sys
from datetime import datetime

## Constants
server = '172.31.119.50'
database = 'Bantotal'
username = 'usr_motor'
password = 'MTM.m0t0R2024'

conn_str = (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    f'SERVER={server};DATABASE={database};UID={username};PWD={password}'
)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

## No arguments: show help table and write to CSV
if len(sys.argv) < 3:
    print("\n🛈 Usage: python script.py <SQL_QUERY> <RECORD_NUMBER>\n")
    help_headers = ["Example SQL Query", "Description"]
    help_data = [
        ["SELECT TOP 5 * FROM fsd010", "Get first 5 records from fsd010"],
        ["SELECT * FROM customers WHERE country = 'USA'", "All US customers"],
        ["SELECT * FROM table", "Full table export"],
    ]

    # Print help table
    print(f"{help_headers[0]:<50} {help_headers[1]}")
    print("-" * 80)
    for row in help_data:
        print(f"{row[0]:<50} {row[1]}")

    # Save help as CSV
    help_file = f"query_help_{timestamp}.csv"
    with open(help_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(help_headers)
        writer.writerows(help_data)

    print(f"\nℹ️ Help saved to '{help_file}'")
    sys.exit(0)

## Read and validate arguments
sql_query = sys.argv[1].strip()
if not sql_query:
    print("Error: SQL query cannot be empty.")
    sys.exit(1)

try:
    record_number = int(sys.argv[2])
    if record_number <= 0:
        raise ValueError
except ValueError:
    print("The record number must be a positive integer.")
    sys.exit(1)

## Output filenames with timestamp
txt_file = f"record_{record_number}_{timestamp}.txt"
csv_file = f"record_{record_number}_{timestamp}.csv"

## Main
with pyodbc.connect(conn_str) as conn:
    cursor = conn.cursor()
    cursor.execute(sql_query)

    rows = cursor.fetchall()

    if not cursor.description:
        print("Error: The query returned no columns (possibly an UPDATE/DELETE statement).")
        sys.exit(1)

    if not rows:
        print("Error: The query returned no records.")
        sys.exit(1)

    if len(rows) < record_number:
        print(f"Error: Only {len(rows)} records returned. Record #{record_number} is out of range.")
        sys.exit(1)

    selected_row = rows[record_number - 1]
    column_info = cursor.description

    headers = [
        "Field_name", "type_code", "display_size", "internal_size",
        "precision", "scale", "null_ok", "Value"
    ]

    table_rows = []
    for i, col in enumerate(column_info):
        row_data = [
            col[0],                # Field name
            str(col[1]),           # Python type code
            col[2],                # Display size
            col[3],                # Internal size
            col[4],                # Precision
            col[5],                # Scale
            col[6],                # Null OK
            selected_row[i]        # Value
        ]
        table_rows.append(row_data)

    # Print to console
    print(f"\nFull metadata and values for record #{record_number}:\n")
    print(f"{headers[0]:<20} {headers[1]:<30} {headers[2]:<13} {headers[3]:<14} "
          f"{headers[4]:<10} {headers[5]:<7} {headers[6]:<8} {headers[7]}")
    print("-" * 130)
    for row in table_rows:
        print(f"{str(row[0]):<20} {str(row[1]):<30} {str(row[2]):<13} {str(row[3]):<14} "
              f"{str(row[4]):<10} {str(row[5]):<7} {str(row[6]):<8} {row[7]}")

    # Save to .txt
    with open(txt_file, 'w', encoding='utf-8') as f:
        f.write(f"Full metadata and values for record #{record_number}:\n\n")
        f.write(f"{headers[0]:<20} {headers[1]:<30} {headers[2]:<13} {headers[3]:<14} "
                f"{headers[4]:<10} {headers[5]:<7} {headers[6]:<8} {headers[7]}\n")
        f.write("-" * 130 + "\n")
        for row in table_rows:
            f.write(f"{str(row[0]):<20} {str(row[1]):<30} {str(row[2]):<13} {str(row[3]):<14} "
                    f"{str(row[4]):<10} {str(row[5]):<7} {str(row[6]):<8} {row[7]}\n")

    # Save to .csv
    with open(csv_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(headers)
        writer.writerows(table_rows)

    print(f"\n✔️ Output exported to:\n- {txt_file}\n- {csv_file}")
    input("\nPress 'Enter' to finish.")
```

---

### ✅ Ejemplo de uso desde terminal:

```bash
python script.py "SELECT TOP 5 * FROM fsd010" 3
```

Esto mostrará e imprimirá la metadata del 3er registro devuelto por el query, con salida timestamp.

---

¿Quieres que la salida también se genere como HTML o Markdown en el futuro para presentación?


Perfecto. A continuación te muestro cómo **extender tu script** para que además de `.txt` y `.csv`, también genere:

---

### ✅ Salidas adicionales:

* 📄 `record_<n>_<timestamp>.html`
* 📘 `record_<n>_<timestamp>.md` (Markdown)

---

### 🆕 Código adicional para incluir después de guardar el `.csv`:

Agrega lo siguiente justo después del bloque que guarda el `.csv` en tu script actual:

```python
    # --- Save to .html ---
    html_file = f"record_{record_number}_{timestamp}.html"
    with open(html_file, 'w', encoding='utf-8') as f:
        f.write(f"<h2>Full metadata and values for record #{record_number}</h2>\n")
        f.write("<table border='1' cellspacing='0' cellpadding='5'>\n")
        f.write("<tr>" + "".join(f"<th>{h}</th>" for h in headers) + "</tr>\n")
        for row in table_rows:
            f.write("<tr>" + "".join(f"<td>{str(cell)}</td>" for cell in row) + "</tr>\n")
        f.write("</table>\n")

    # --- Save to .md ---
    md_file = f"record_{record_number}_{timestamp}.md"
    with open(md_file, 'w', encoding='utf-8') as f:
        f.write(f"# Full metadata and values for record #{record_number}\n\n")
        f.write("| " + " | ".join(headers) + " |\n")
        f.write("|" + "|".join(["-" * len(h) for h in headers]) + "|\n")
        for row in table_rows:
            f.write("| " + " | ".join(str(cell) for cell in row) + " |\n")
```

---

### 🔚 Resultado final:

Cuando ejecutes por ejemplo:

```bash
python script.py "SELECT TOP 5 * FROM fsd010" 3
```

El script generará:

* `record_3_20250506_151300.txt`
* `record_3_20250506_151300.csv`
* `record_3_20250506_151300.html`
* `record_3_20250506_151300.md`

Todos los archivos estarán correctamente formateados y listos para uso en reportes, navegadores o documentos.

---

¿Quieres que empaquemos todos estos archivos en un `.zip` automáticamente al final?


Aquí tienes el **script completo, corregido y mejorado** con todo lo que pediste:

---

### ✅ Funcionalidades incluidas:

* 🔢 **Primer argumento**: Query SQL a ejecutar
* 🔢 **Segundo argumento**: Número de registro deseado
* ❌ Si no hay argumentos: muestra la tabla descriptiva de `cursor.description` y la guarda como `.csv`
* 🕒 Archivos `.txt` y `.csv` incluyen **fecha y hora** para evitar sobrescritura

---

### 🆕 Código actualizado (versión final profesional):

```python
import pyodbc
import csv
import sys
from datetime import datetime

# Timestamp for filenames
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Show cursor.description structure if no arguments are passed
if len(sys.argv) < 3:
    headers = ["Index", "Element", "Meaning"]
    table_data = [
        [0, "`name`", "Column name (string)"],
        [1, "`type_code`", "Data type code (int, str, etc.) depending on driver"],
        [2, "`display_size`", "Max display width (usually None in pyodbc)"],
        [3, "`internal_size`", "Internal size in bytes (e.g., 4 for INT)"],
        [4, "`precision`", "Total digits for numeric fields"],
        [5, "`scale`", "Digits to the right of decimal point"],
        [6, "`null_ok`", "True if the column accepts NULLs"]
    ]

    print("\n📘 Structure of cursor.description:\n")
    print(f"{headers[0]:<8} {headers[1]:<20} {headers[2]}")
    print("-" * 70)
    for row in table_data:
        print(f"{str(row[0]):<8} {row[1]:<20} {row[2]}")

    csv_file = f"cursor_description_{timestamp}.csv"
    with open(csv_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(headers)
        writer.writerows(table_data)

    print(f"\n✔️ Structure saved to '{csv_file}'")
    sys.exit(0)

# Read command-line arguments
sql_query = sys.argv[1]
try:
    record_number = int(sys.argv[2])
    if record_number <= 0:
        raise ValueError
except ValueError:
    print("❌ The record number must be a positive integer.")
    sys.exit(1)

# Connection parameters
server = '172.31.119.50'
database = 'Bantotal'
username = 'usr_motor'
password = 'MTM.m0t0R2024'

conn_str = (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    f'SERVER={server};DATABASE={database};UID={username};PWD={password}'
)

# Output filenames with timestamp
txt_file = f"record_{record_number}_{timestamp}.txt"
csv_file = f"record_{record_number}_{timestamp}.csv"

# Execute query and export result
with pyodbc.connect(conn_str) as conn:
    cursor = conn.cursor()
    cursor.execute(sql_query)

    rows = cursor.fetchall()
    if not cursor.description:
        print("❌ Query returned no columns. Check for UPDATE/DELETE statements.")
        sys.exit(1)

    if len(rows) < record_number:
        print(f"❌ Only {len(rows)} records found. Record #{record_number} is out of range.")
        sys.exit(1)

    selected_row = rows[record_number - 1]
    column_info = cursor.description

    headers = [
        "Field_name", "type_code", "display_size", "internal_size",
        "precision", "scale", "null_ok", "Value"
    ]

    table_rows = []
    for i, col in enumerate(column_info):
        row_data = [
            col[0],        # name
            str(col[1]),   # type_code
            col[2],        # display_size
            col[3],        # internal_size
            col[4],        # precision
            col[5],        # scale
            col[6],        # null_ok
            selected_row[i]  # value
        ]
        table_rows.append(row_data)

    # Print to console
    print(f"\n📄 Full information for record #{record_number}:\n")
    print(f"{headers[0]:<20} {headers[1]:<30} {headers[2]:<13} {headers[3]:<14} "
          f"{headers[4]:<10} {headers[5]:<7} {headers[6]:<8} {headers[7]}")
    print("-" * 130)
    for row in table_rows:
        print(f"{str(row[0]):<20} {str(row[1]):<30} {str(row[2]):<13} {str(row[3]):<14} "
              f"{str(row[4]):<10} {str(row[5]):<7} {str(row[6]):<8} {row[7]}")

    # Save to .txt
    with open(txt_file, 'w', encoding='utf-8') as f:
        f.write(f"Full information for record #{record_number}:\n\n")
        f.write(f"{headers[0]:<20} {headers[1]:<30} {headers[2]:<13} {headers[3]:<14} "
                f"{headers[4]:<10} {headers[5]:<7} {headers[6]:<8} {headers[7]}\n")
        f.write("-" * 130 + "\n")
        for row in table_rows:
            f.write(f"{str(row[0]):<20} {str(row[1]):<30} {str(row[2]):<13} {str(row[3]):<14} "
                    f"{str(row[4]):<10} {str(row[5]):<7} {str(row[6]):<8} {row[7]}\n")

    # Save to .csv
    with open(csv_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(headers)
        writer.writerows(table_rows)

    print(f"\n✅ Output saved to:\n- {txt_file}\n- {csv_file}")
    input("\nPress 'Enter' to finish.")
```

---

### ▶️ Ejemplo de ejecución:

```bash
python script.py "SELECT TOP 9 * FROM fsd010" 3
```

Esto seleccionará el **3er registro** del resultado y generará:

* `record_3_YYYYMMDD_HHMMSS.txt`
* `record_3_YYYYMMDD_HHMMSS.csv`

---

¿Te gustaría que también se exporten los datos a Excel (`.xlsx`) automáticamente?


### 5 var
Puedes modificar el código anterior para que en vez de tomar el query del primer argumento lo tome de un archivo de texto llamado reganalys.qry y que el primer argumento sea el número de registro deseado?

Claro. Aquí tienes el código **modificado** para que:

---

### ✅ Cambios solicitados:

* El **query SQL** se lee desde un archivo llamado `reganalys.qry` (en el mismo directorio)
* El **primer argumento** (`sys.argv[1]`) ahora es el número de registro deseado
* Todo lo demás (validación, exportación, fecha/hora en archivos) se mantiene

---

### 🆕 Código actualizado:

```python
import pyodbc
import csv
import sys
from datetime import datetime
import os

# Timestamp for filenames
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Query file
query_file = "reganalys.qry"

# Check if query file exists
if not os.path.exists(query_file):
    print(f"❌ Query file '{query_file}' not found.")
    sys.exit(1)

# Read query from file
with open(query_file, 'r', encoding='utf-8') as f:
    sql_query = f.read().strip()

if not sql_query:
    print("❌ The query in 'reganalys.qry' is empty.")
    sys.exit(1)

# No arguments: show structure of cursor.description and exit
if len(sys.argv) < 2:
    headers = ["Index", "Element", "Meaning"]
    table_data = [
        [0, "`name`", "Column name (string)"],
        [1, "`type_code`", "Data type code (int, str, etc.) depending on driver"],
        [2, "`display_size`", "Max display width (usually None in pyodbc)"],
        [3, "`internal_size`", "Internal size in bytes (e.g., 4 for INT)"],
        [4, "`precision`", "Total digits for numeric fields"],
        [5, "`scale`", "Digits to the right of decimal point"],
        [6, "`null_ok`", "True if the column accepts NULLs"]
    ]

    print("\n📘 Structure of cursor.description:\n")
    print(f"{headers[0]:<8} {headers[1]:<20} {headers[2]}")
    print("-" * 70)
    for row in table_data:
        print(f"{str(row[0]):<8} {row[1]:<20} {row[2]}")

    csv_file = f"cursor_description_{timestamp}.csv"
    with open(csv_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(headers)
        writer.writerows(table_data)

    print(f"\n✔️ Structure saved to '{csv_file}'")
    sys.exit(0)

# Read and validate record number from argument
try:
    record_number = int(sys.argv[1])
    if record_number <= 0:
        raise ValueError
except ValueError:
    print("❌ The record number must be a positive integer.")
    sys.exit(1)

# Connection parameters
server = '172.31.119.50'
database = 'Bantotal'
username = 'usr_motor'
password = 'MTM.m0t0R2024'

conn_str = (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    f'SERVER={server};DATABASE={database};UID={username};PWD={password}'
)

# Output filenames
txt_file = f"record_{record_number}_{timestamp}.txt"
csv_file = f"record_{record_number}_{timestamp}.csv"

# Main execution
with pyodbc.connect(conn_str) as conn:
    cursor = conn.cursor()
    cursor.execute(sql_query)

    rows = cursor.fetchall()
    if not cursor.description:
        print("❌ Query returned no columns. Check for UPDATE/DELETE statements.")
        sys.exit(1)

    if len(rows) < record_number:
        print(f"❌ Only {len(rows)} records found. Record #{record_number} is out of range.")
        sys.exit(1)

    selected_row = rows[record_number - 1]
    column_info = cursor.description

    headers = [
        "Field_name", "type_code", "display_size", "internal_size",
        "precision", "scale", "null_ok", "Value"
    ]

    table_rows = []
    for i, col in enumerate(column_info):
        row_data = [
            col[0],        # name
            str(col[1]),   # type_code
            col[2],        # display_size
            col[3],        # internal_size
            col[4],        # precision
            col[5],        # scale
            col[6],        # null_ok
            selected_row[i]  # value
        ]
        table_rows.append(row_data)

    # Print to console
    print(f"\n📄 Full information for record #{record_number}:\n")
    print(f"{headers[0]:<20} {headers[1]:<30} {headers[2]:<13} {headers[3]:<14} "
          f"{headers[4]:<10} {headers[5]:<7} {headers[6]:<8} {headers[7]}")
    print("-" * 130)
    for row in table_rows:
        print(f"{str(row[0]):<20} {str(row[1]):<30} {str(row[2]):<13} {str(row[3]):<14} "
              f"{str(row[4]):<10} {str(row[5]):<7} {str(row[6]):<8} {row[7]}")

    # Save to .txt
    with open(txt_file, 'w', encoding='utf-8') as f:
        f.write(f"Full information for record #{record_number}:\n\n")
        f.write(f"{headers[0]:<20} {headers[1]:<30} {headers[2]:<13} {headers[3]:<14} "
                f"{headers[4]:<10} {headers[5]:<7} {headers[6]:<8} {headers[7]}\n")
        f.write("-" * 130 + "\n")
        for row in table_rows:
            f.write(f"{str(row[0]):<20} {str(row[1]):<30} {str(row[2]):<13} {str(row[3]):<14} "
                    f"{str(row[4]):<10} {str(row[5]):<7} {str(row[6]):<8} {row[7]}\n")

    # Save to .csv
    with open(csv_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(headers)
        writer.writerows(table_rows)

    print(f"\n✅ Output saved to:\n- {txt_file}\n- {csv_file}")
    input("\nPress 'Enter' to finish.")
```

---

### 📌 ¿Cómo usarlo?

1. Crea un archivo de texto llamado `reganalys.qry` con tu SQL (por ejemplo: `SELECT TOP 9 * FROM fsd010`)
2. Ejecuta el script pasando solo el número de registro como argumento:

```bash
python script.py 3
```

Esto tomará el 3er registro del resultado del query dentro de `reganalys.qry`.

---

¿Quieres que además exporte automáticamente como `.html` y `.md` en esta versión también?


Perfecto. A continuación tienes el **script completo y final**, que:

---

### ✅ Incluye:

* 📄 Lee el **SQL** desde `reganalys.qry`
* 🔢 Usa el **número de registro** como **primer argumento**
* 💾 Exporta los resultados a `.txt`, `.csv`, `.html` y `.md`
* 🕒 Archivos nombrados con **fecha y hora** para evitar sobrescrituras
* 📘 Si no se pasan argumentos, muestra una tabla explicativa de `cursor.description` y la guarda como `.csv`

---

### 🆕 Script Final (versión extendida):

```python
import pyodbc
import csv
import sys
from datetime import datetime
import os

# Timestamp for filenames
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Query file
query_file = "reganalys.qry"

# If no arguments provided, show structure of cursor.description
if len(sys.argv) < 2:
    headers = ["Index", "Element", "Meaning"]
    table_data = [
        [0, "`name`", "Column name (string)"],
        [1, "`type_code`", "Data type code (int, str, etc.) depending on driver"],
        [2, "`display_size`", "Max display width (usually None in pyodbc)"],
        [3, "`internal_size`", "Internal size in bytes (e.g., 4 for INT)"],
        [4, "`precision`", "Total digits for numeric fields"],
        [5, "`scale`", "Digits to the right of decimal point"],
        [6, "`null_ok`", "True if the column accepts NULLs"]
    ]

    print("\n📘 Structure of cursor.description:\n")
    print(f"{headers[0]:<8} {headers[1]:<20} {headers[2]}")
    print("-" * 70)
    for row in table_data:
        print(f"{str(row[0]):<8} {row[1]:<20} {row[2]}")

    csv_file = f"cursor_description_{timestamp}.csv"
    with open(csv_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(headers)
        writer.writerows(table_data)

    print(f"\n✔️ Structure saved to '{csv_file}'")
    sys.exit(0)

# Read and validate record number
try:
    record_number = int(sys.argv[1])
    if record_number <= 0:
        raise ValueError
except ValueError:
    print("❌ The record number must be a positive integer.")
    sys.exit(1)

# Load query from file
if not os.path.exists(query_file):
    print(f"❌ Query file '{query_file}' not found.")
    sys.exit(1)

with open(query_file, 'r', encoding='utf-8') as f:
    sql_query = f.read().strip()

if not sql_query:
    print("❌ The query in 'reganalys.qry' is empty.")
    sys.exit(1)

# Connection settings
server = '172.31.119.50'
database = 'Bantotal'
username = 'usr_motor'
password = 'MTM.m0t0R2024'

conn_str = (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    f'SERVER={server};DATABASE={database};UID={username};PWD={password}'
)

# Output filenames
base_filename = f"record_{record_number}_{timestamp}"
txt_file = f"{base_filename}.txt"
csv_file = f"{base_filename}.csv"
html_file = f"{base_filename}.html"
md_file = f"{base_filename}.md"

# Main process
with pyodbc.connect(conn_str) as conn:
    cursor = conn.cursor()
    cursor.execute(sql_query)

    rows = cursor.fetchall()
    if not cursor.description:
        print("❌ Query returned no columns.")
        sys.exit(1)

    if len(rows) < record_number:
        print(f"❌ Only {len(rows)} records found. Record #{record_number} is out of range.")
        sys.exit(1)

    selected_row = rows[record_number - 1]
    column_info = cursor.description

    headers = [
        "Field_name", "type_code", "display_size", "internal_size",
        "precision", "scale", "null_ok", "Value"
    ]

    table_rows = []
    for i, col in enumerate(column_info):
        row_data = [
            col[0],
            str(col[1]),
            col[2],
            col[3],
            col[4],
            col[5],
            col[6],
            selected_row[i]
        ]
        table_rows.append(row_data)

    # Console output
    print(f"\n📄 Full information for record #{record_number}:\n")
    print(f"{headers[0]:<20} {headers[1]:<30} {headers[2]:<13} {headers[3]:<14} "
          f"{headers[4]:<10} {headers[5]:<7} {headers[6]:<8} {headers[7]}")
    print("-" * 130)
    for row in table_rows:
        print(f"{str(row[0]):<20} {str(row[1]):<30} {str(row[2]):<13} {str(row[3]):<14} "
              f"{str(row[4]):<10} {str(row[5]):<7} {str(row[6]):<8} {row[7]}")

    # Save to .txt
    with open(txt_file, 'w', encoding='utf-8') as f:
        f.write(f"Full information for record #{record_number}:\n\n")
        f.write(f"{headers[0]:<20} {headers[1]:<30} {headers[2]:<13} {headers[3]:<14} "
                f"{headers[4]:<10} {headers[5]:<7} {headers[6]:<8} {headers[7]}\n")
        f.write("-" * 130 + "\n")
        for row in table_rows:
            f.write(f"{str(row[0]):<20} {str(row[1]):<30} {str(row[2]):<13} {str(row[3]):<14} "
                    f"{str(row[4]):<10} {str(row[5]):<7} {str(row[6]):<8} {row[7]}\n")

    # Save to .csv
    with open(csv_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(headers)
        writer.writerows(table_rows)

    # Save to .html
    with open(html_file, 'w', encoding='utf-8') as f:
        f.write(f"<h2>Full metadata and values for record #{record_number}</h2>\n")
        f.write("<table border='1' cellspacing='0' cellpadding='5'>\n")
        f.write("<tr>" + "".join(f"<th>{h}</th>" for h in headers) + "</tr>\n")
        for row in table_rows:
            f.write("<tr>" + "".join(f"<td>{str(cell)}</td>" for cell in row) + "</tr>\n")
        f.write("</table>\n")

    # Save to .md
    with open(md_file, 'w', encoding='utf-8') as f:
        f.write(f"# Full metadata and values for record #{record_number}\n\n")
        f.write("| " + " | ".join(headers) + " |\n")
        f.write("|" + "|".join(["-" * len(h) for h in headers]) + "|\n")
        for row in table_rows:
            f.write("| " + " | ".join(str(cell) for cell in row) + " |\n")

    print(f"\n✅ Output saved to:")
    print(f"  - {txt_file}\n  - {csv_file}\n  - {html_file}\n  - {md_file}")
    input("\nPress 'Enter' to finish.")
```

---

### ▶️ Uso:

1. Guarda tu SQL en un archivo `reganalys.qry`
2. Ejecuta el script desde terminal:

```bash
python script.py 2
```

Esto selecciona el segundo registro del resultado de tu query y genera `.txt`, `.csv`, `.html` y `.md`.

---

¿Quieres que también se genere un archivo `.xlsx` (Excel) opcionalmente?


## 6 Con formatos de salida opcionales

Perfecto. Como estás interesado en que los **formatos de salida sean opcionales** y definidos por argumentos del script, te dejo el código actualizado para lograrlo.

Este script ahora:

---

### ✅ Hace lo siguiente:

* Toma el **número de registro** como primer argumento (obligatorio)
* Toma **opcionalmente** una lista de formatos (`txt`, `csv`, `html`, `md`, `xlsx`) como argumentos adicionales
* Por defecto genera `.txt` y `.csv` si no se indica nada
* Genera archivos con **marca de tiempo**
* Lee el query desde `reganalys.qry`

---

### 🧩 Código actualizado:

```python
import pyodbc
import csv
import sys
import os
from datetime import datetime
import argparse

try:
    import openpyxl
    from openpyxl import Workbook
except ImportError:
    openpyxl = None  # Will be checked before xlsx export

# Argument parsing
parser = argparse.ArgumentParser(description="Fetch and export a selected record from a SQL Server query result.")
parser.add_argument("record_number", type=int, help="The record number to retrieve (1-based index).")
parser.add_argument("--formats", nargs="+", choices=["txt", "csv", "html", "md", "xlsx"], default=["txt", "csv"],
                    help="List of output formats to generate (default: txt csv)")
args = parser.parse_args()

record_number = args.record_number
output_formats = args.formats
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
base_filename = f"record_{record_number}_{timestamp}"

# Load SQL query
query_file = "reganalys.qry"
if not os.path.exists(query_file):
    print(f"❌ Query file '{query_file}' not found.")
    sys.exit(1)

with open(query_file, 'r', encoding='utf-8') as f:
    sql_query = f.read().strip()

if not sql_query:
    print("❌ SQL query file is empty.")
    sys.exit(1)

# Connection details
server = '172.31.119.50'
database = 'Bantotal'
username = 'usr_motor'
password = 'MTM.m0t0R2024'

conn_str = (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    f'SERVER={server};DATABASE={database};UID={username};PWD={password}'
)

# Connect and execute
with pyodbc.connect(conn_str) as conn:
    cursor = conn.cursor()
    cursor.execute(sql_query)

    rows = cursor.fetchall()
    if not cursor.description:
        print("❌ Query returned no columns.")
        sys.exit(1)

    if len(rows) < record_number:
        print(f"❌ Only {len(rows)} rows available. Requested record {record_number} is out of range.")
        sys.exit(1)

    selected_row = rows[record_number - 1]
    column_info = cursor.description

    headers = [
        "Field_name", "type_code", "display_size", "internal_size",
        "precision", "scale", "null_ok", "Value"
    ]

    table_rows = [
        [
            col[0], str(col[1]), col[2], col[3],
            col[4], col[5], col[6], selected_row[i]
        ]
        for i, col in enumerate(column_info)
    ]

# Export functions
def save_txt(path):
    with open(path, 'w', encoding='utf-8') as f:
        f.write(f"Full information for record #{record_number}:\n\n")
        f.write(f"{headers[0]:<20} {headers[1]:<30} {headers[2]:<13} {headers[3]:<14} "
                f"{headers[4]:<10} {headers[5]:<7} {headers[6]:<8} {headers[7]}\n")
        f.write("-" * 130 + "\n")
        for row in table_rows:
            f.write(f"{str(row[0]):<20} {str(row[1]):<30} {str(row[2]):<13} {str(row[3]):<14} "
                    f"{str(row[4]):<10} {str(row[5]):<7} {str(row[6]):<8} {row[7]}\n")

def save_csv(path):
    with open(path, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(headers)
        writer.writerows(table_rows)

def save_html(path):
    with open(path, 'w', encoding='utf-8') as f:
        f.write(f"<h2>Record #{record_number}</h2>\n<table border='1'>\n<tr>")
        f.write("".join(f"<th>{h}</th>" for h in headers) + "</tr>\n")
        for row in table_rows:
            f.write("<tr>" + "".join(f"<td>{str(cell)}</td>" for cell in row) + "</tr>\n")
        f.write("</table>\n")

def save_md(path):
    with open(path, 'w', encoding='utf-8') as f:
        f.write(f"# Record #{record_number}\n\n")
        f.write("| " + " | ".join(headers) + " |\n")
        f.write("|" + "|".join(["-" * len(h) for h in headers]) + "|\n")
        for row in table_rows:
            f.write("| " + " | ".join(str(cell) for cell in row) + " |\n")

def save_xlsx(path):
    if not openpyxl:
        print("⚠️ Cannot save to Excel: openpyxl is not installed.")
        return
    wb = Workbook()
    ws = wb.active
    ws.title = f"Record_{record_number}"
    ws.append(headers)
    for row in table_rows:
        ws.append(row)
    wb.save(path)

# Export based on selected formats
print(f"\n📄 Exporting record #{record_number}...\n")

if "txt" in output_formats:
    save_txt(f"{base_filename}.txt")
if "csv" in output_formats:
    save_csv(f"{base_filename}.csv")
if "html" in output_formats:
    save_html(f"{base_filename}.html")
if "md" in output_formats:
    save_md(f"{base_filename}.md")
if "xlsx" in output_formats:
    save_xlsx(f"{base_filename}.xlsx")

print("✅ Export completed.")
```

---

### ✅ Cómo usarlo:

```bash
# Ejemplo con 3er registro y exportación en HTML y Excel
python script.py 3 --formats html xlsx

# Exportar en todos los formatos:
python script.py 1 --formats txt csv html md xlsx
```

> 💡 Si no pasas `--formats`, genera `.txt` y `.csv` por defecto.

---

¿Te gustaría que agregue también una opción para exportar solo ciertas columnas?


## 7 versión segura del script que elimina las credenciales

Perfecto. A continuación te dejo una **versión segura del script** que **elimina las credenciales** y cualquier dato sensible. Esta versión:

---

### ✅ Cambios realizados:

* Elimina la IP del servidor, base de datos, usuario y contraseña
* Usa variables de entorno para cargar las credenciales de forma segura
* Incluye instrucciones para configurar el entorno

---

### 🔐 Versión Segura del Script

```python
import pyodbc
import csv
import sys
import os
from datetime import datetime
import argparse

try:
    import openpyxl
    from openpyxl import Workbook
except ImportError:
    openpyxl = None

# Argument parsing
parser = argparse.ArgumentParser(description="Export selected record from a SQL Server query result.")
parser.add_argument("record_number", type=int, help="The record number to retrieve (1-based index).")
parser.add_argument("--formats", nargs="+", choices=["txt", "csv", "html", "md", "xlsx"], default=["txt", "csv"],
                    help="Output formats (default: txt csv)")
args = parser.parse_args()

record_number = args.record_number
output_formats = args.formats
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
base_filename = f"record_{record_number}_{timestamp}"

# Load SQL query from file
query_file = "reganalys.qry"
if not os.path.exists(query_file):
    print(f"❌ Query file '{query_file}' not found.")
    sys.exit(1)

with open(query_file, 'r', encoding='utf-8') as f:
    sql_query = f.read().strip()

if not sql_query:
    print("❌ SQL query file is empty.")
    sys.exit(1)

# Load DB credentials from environment variables
server = os.getenv("DB_SERVER")
database = os.getenv("DB_NAME")
username = os.getenv("DB_USER")
password = os.getenv("DB_PASS")

if not all([server, database, username, password]):
    print("❌ One or more database credentials are missing. Set DB_SERVER, DB_NAME, DB_USER, DB_PASS.")
    sys.exit(1)

conn_str = (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    f'SERVER={server};DATABASE={database};UID={username};PWD={password}'
)

with pyodbc.connect(conn_str) as conn:
    cursor = conn.cursor()
    cursor.execute(sql_query)

    rows = cursor.fetchall()
    if not cursor.description:
        print("❌ Query returned no columns.")
        sys.exit(1)

    if len(rows) < record_number:
        print(f"❌ Only {len(rows)} rows available. Requested record {record_number} is out of range.")
        sys.exit(1)

    selected_row = rows[record_number - 1]
    column_info = cursor.description

    headers = [
        "Field_name", "type_code", "display_size", "internal_size",
        "precision", "scale", "null_ok", "Value"
    ]

    table_rows = [
        [
            col[0], str(col[1]), col[2], col[3],
            col[4], col[5], col[6], selected_row[i]
        ]
        for i, col in enumerate(column_info)
    ]

# Export functions
def save_txt(path):
    with open(path, 'w', encoding='utf-8') as f:
        f.write(f"Full information for record #{record_number}:\n\n")
        f.write(f"{headers[0]:<20} {headers[1]:<30} {headers[2]:<13} {headers[3]:<14} "
                f"{headers[4]:<10} {headers[5]:<7} {headers[6]:<8} {headers[7]}\n")
        f.write("-" * 130 + "\n")
        for row in table_rows:
            f.write(f"{str(row[0]):<20} {str(row[1]):<30} {str(row[2]):<13} {str(row[3]):<14} "
                    f"{str(row[4]):<10} {str(row[5]):<7} {str(row[6]):<8} {row[7]}\n")

def save_csv(path):
    with open(path, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(headers)
        writer.writerows(table_rows)

def save_html(path):
    with open(path, 'w', encoding='utf-8') as f:
        f.write(f"<h2>Record #{record_number}</h2>\n<table border='1'>\n<tr>")
        f.write("".join(f"<th>{h}</th>" for h in headers) + "</tr>\n")
        for row in table_rows:
            f.write("<tr>" + "".join(f"<td>{str(cell)}</td>" for cell in row) + "</tr>\n")
        f.write("</table>\n")

def save_md(path):
    with open(path, 'w', encoding='utf-8') as f:
        f.write(f"# Record #{record_number}\n\n")
        f.write("| " + " | ".join(headers) + " |\n")
        f.write("|" + "|".join(["-" * len(h) for h in headers]) + "|\n")
        for row in table_rows:
            f.write("| " + " | ".join(str(cell) for cell in row) + " |\n")

def save_xlsx(path):
    if not openpyxl:
        print("⚠️ Cannot save to Excel: openpyxl is not installed.")
        return
    wb = Workbook()
    ws = wb.active
    ws.title = f"Record_{record_number}"
    ws.append(headers)
    for row in table_rows:
        ws.append(row)
    wb.save(path)

# Export
if "txt" in output_formats:
    save_txt(f"{base_filename}.txt")
if "csv" in output_formats:
    save_csv(f"{base_filename}.csv")
if "html" in output_formats:
    save_html(f"{base_filename}.html")
if "md" in output_formats:
    save_md(f"{base_filename}.md")
if "xlsx" in output_formats:
    save_xlsx(f"{base_filename}.xlsx")

print("\n✅ Export complete.")
```

---

### 🛡️ Cómo configurar el entorno de forma segura:

Antes de ejecutar el script, configura las variables de entorno:

#### En Linux / macOS:

```bash
export DB_SERVER=your_server
export DB_NAME=your_database
export DB_USER=your_user
export DB_PASS=your_password
```

#### En Windows (CMD):

```cmd
set DB_SERVER=your_server
set DB_NAME=your_database
set DB_USER=your_user
set DB_PASS=your_password
```

---

¿Te gustaría que también cree un archivo `.env` y lo lea automáticamente si existe?


Perfecto. A continuación te dejo el script **actualizado** para que:

---

### ✅ Ahora también:

* Cargue las variables de entorno desde un archivo `.env` (si existe)
* Priorice valores del sistema, pero use `.env` como respaldo
* Use `python-dotenv` si está instalado (te mostraré cómo instalarlo)

---

### 💾 Paso 1: Crear archivo `.env`

Guarda esto como `.env` en el mismo directorio del script:

```
DB_SERVER=172.31.119.50
DB_NAME=Bantotal
DB_USER=usr_motor
DB_PASS=MTM.m0t0R2024
```

---

### 📦 Paso 2: Instalar `python-dotenv` (si no lo tenés)

```bash
pip install python-dotenv
```

---

### 🧩 Paso 3: Script final con soporte `.env`

```python
import pyodbc
import csv
import sys
import os
from datetime import datetime
import argparse

try:
    from dotenv import load_dotenv
    dotenv_loaded = True
except ImportError:
    dotenv_loaded = False

try:
    import openpyxl
    from openpyxl import Workbook
except ImportError:
    openpyxl = None

# Load environment variables from .env if available
if dotenv_loaded and os.path.exists(".env"):
    load_dotenv(dotenv_path=".env")

# Argument parsing
parser = argparse.ArgumentParser(description="Export selected record from a SQL Server query result.")
parser.add_argument("record_number", type=int, help="The record number to retrieve (1-based index).")
parser.add_argument("--formats", nargs="+", choices=["txt", "csv", "html", "md", "xlsx"], default=["txt", "csv"],
                    help="Output formats (default: txt csv)")
args = parser.parse_args()

record_number = args.record_number
output_formats = args.formats
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
base_filename = f"record_{record_number}_{timestamp}"

# Load SQL query from file
query_file = "reganalys.qry"
if not os.path.exists(query_file):
    print(f"❌ Query file '{query_file}' not found.")
    sys.exit(1)

with open(query_file, 'r', encoding='utf-8') as f:
    sql_query = f.read().strip()

if not sql_query:
    print("❌ SQL query file is empty.")
    sys.exit(1)

# Load DB credentials from environment variables or .env
server = os.getenv("DB_SERVER")
database = os.getenv("DB_NAME")
username = os.getenv("DB_USER")
password = os.getenv("DB_PASS")

if not all([server, database, username, password]):
    print("❌ Missing DB credentials. Please set environment variables or define them in a .env file.")
    sys.exit(1)

conn_str = (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    f'SERVER={server};DATABASE={database};UID={username};PWD={password}'
)

with pyodbc.connect(conn_str) as conn:
    cursor = conn.cursor()
    cursor.execute(sql_query)

    rows = cursor.fetchall()
    if not cursor.description:
        print("❌ Query returned no columns.")
        sys.exit(1)

    if len(rows) < record_number:
        print(f"❌ Only {len(rows)} rows available. Requested record {record_number} is out of range.")
        sys.exit(1)

    selected_row = rows[record_number - 1]
    column_info = cursor.description

    headers = [
        "Field_name", "type_code", "display_size", "internal_size",
        "precision", "scale", "null_ok", "Value"
    ]

    table_rows = [
        [
            col[0], str(col[1]), col[2], col[3],
            col[4], col[5], col[6], selected_row[i]
        ]
        for i, col in enumerate(column_info)
    ]

# Export functions
def save_txt(path):
    with open(path, 'w', encoding='utf-8') as f:
        f.write(f"Full information for record #{record_number}:\n\n")
        f.write(f"{headers[0]:<20} {headers[1]:<30} {headers[2]:<13} {headers[3]:<14} "
                f"{headers[4]:<10} {headers[5]:<7} {headers[6]:<8} {headers[7]}\n")
        f.write("-" * 130 + "\n")
        for row in table_rows:
            f.write(f"{str(row[0]):<20} {str(row[1]):<30} {str(row[2]):<13} {str(row[3]):<14} "
                    f"{str(row[4]):<10} {str(row[5]):<7} {str(row[6]):<8} {row[7]}\n")

def save_csv(path):
    with open(path, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(headers)
        writer.writerows(table_rows)

def save_html(path):
    with open(path, 'w', encoding='utf-8') as f:
        f.write(f"<h2>Record #{record_number}</h2>\n<table border='1'>\n<tr>")
        f.write("".join(f"<th>{h}</th>" for h in headers) + "</tr>\n")
        for row in table_rows:
            f.write("<tr>" + "".join(f"<td>{str(cell)}</td>" for cell in row) + "</tr>\n")
        f.write("</table>\n")

def save_md(path):
    with open(path, 'w', encoding='utf-8') as f:
        f.write(f"# Record #{record_number}\n\n")
        f.write("| " + " | ".join(headers) + " |\n")
        f.write("|" + "|".join(["-" * len(h) for h in headers]) + "|\n")
        for row in table_rows:
            f.write("| " + " | ".join(str(cell) for cell in row) + " |\n")

def save_xlsx(path):
    if not openpyxl:
        print("⚠️ Cannot save to Excel: openpyxl is not installed.")
        return
    wb = Workbook()
    ws = wb.active
    ws.title = f"Record_{record_number}"
    ws.append(headers)
    for row in table_rows:
        ws.append(row)
    wb.save(path)

# Export
if "txt" in output_formats:
    save_txt(f"{base_filename}.txt")
if "csv" in output_formats:
    save_csv(f"{base_filename}.csv")
if "html" in output_formats:
    save_html(f"{base_filename}.html")
if "md" in output_formats:
    save_md(f"{base_filename}.md")
if "xlsx" in output_formats:
    save_xlsx(f"{base_filename}.xlsx")

print("\n✅ Export complete.")
```

---

¿Te gustaría también que el script cree automáticamente el archivo `.env` con placeholders si no existe?


Perfecto. Aquí tienes la **versión final mejorada del script** que:

---

### ✅ Ahora también:

* Crea un archivo `.env` automáticamente si no existe, con *placeholders* seguros
* Te avisa que debes completarlo antes de ejecutar el script
* Detiene la ejecución si el `.env` no está completo

---

### 🧩 Código Final — Seguro y Automático

```python
import pyodbc
import csv
import sys
import os
from datetime import datetime
import argparse

try:
    from dotenv import load_dotenv
    dotenv_loaded = True
except ImportError:
    dotenv_loaded = False

try:
    import openpyxl
    from openpyxl import Workbook
except ImportError:
    openpyxl = None

# Ensure .env exists
env_file = ".env"
if not os.path.exists(env_file):
    with open(env_file, "w") as f:
        f.write(
            "DB_SERVER=your_server_here\n"
            "DB_NAME=your_database_name\n"
            "DB_USER=your_username\n"
            "DB_PASS=your_password\n"
        )
    print(f"📝 A .env file has been created for you at '{env_file}'.")
    print("❗ Please fill in your database credentials and run the script again.")
    sys.exit(1)

# Load environment variables
if dotenv_loaded:
    load_dotenv(dotenv_path=env_file)

# Read DB credentials
server = os.getenv("DB_SERVER")
database = os.getenv("DB_NAME")
username = os.getenv("DB_USER")
password = os.getenv("DB_PASS")

if not all([server, database, username, password]) or "your_" in "".join([server, database, username, password]):
    print("❌ The .env file exists but contains placeholders. Please update it with real credentials.")
    sys.exit(1)

# Read SQL query from file
query_file = "reganalys.qry"
if not os.path.exists(query_file):
    print(f"❌ Query file '{query_file}' not found.")
    sys.exit(1)

with open(query_file, "r", encoding="utf-8") as f:
    sql_query = f.read().strip()

if not sql_query:
    print("❌ SQL query file is empty.")
    sys.exit(1)

# Parse arguments
parser = argparse.ArgumentParser(description="Export selected record from a SQL Server query result.")
parser.add_argument("record_number", type=int, help="The record number to retrieve (1-based index).")
parser.add_argument("--formats", nargs="+", choices=["txt", "csv", "html", "md", "xlsx"], default=["txt", "csv"],
                    help="Output formats (default: txt csv)")
args = parser.parse_args()

record_number = args.record_number
output_formats = args.formats
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
base_filename = f"record_{record_number}_{timestamp}"

# Connect and execute
conn_str = (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    f'SERVER={server};DATABASE={database};UID={username};PWD={password}'
)

with pyodbc.connect(conn_str) as conn:
    cursor = conn.cursor()
    cursor.execute(sql_query)
    rows = cursor.fetchall()

    if not cursor.description:
        print("❌ Query returned no columns.")
        sys.exit(1)
    if len(rows) < record_number:
        print(f"❌ Only {len(rows)} rows available. Requested record {record_number} is out of range.")
        sys.exit(1)

    selected_row = rows[record_number - 1]
    column_info = cursor.description

    headers = [
        "Field_name", "type_code", "display_size", "internal_size",
        "precision", "scale", "null_ok", "Value"
    ]

    table_rows = [
        [
            col[0], str(col[1]), col[2], col[3],
            col[4], col[5], col[6], selected_row[i]
        ]
        for i, col in enumerate(column_info)
    ]

# Export functions
def save_txt(path):
    with open(path, 'w', encoding='utf-8') as f:
        f.write(f"Full information for record #{record_number}:\n\n")
        f.write(f"{headers[0]:<20} {headers[1]:<30} {headers[2]:<13} {headers[3]:<14} "
                f"{headers[4]:<10} {headers[5]:<7} {headers[6]:<8} {headers[7]}\n")
        f.write("-" * 130 + "\n")
        for row in table_rows:
            f.write(f"{str(row[0]):<20} {str(row[1]):<30} {str(row[2]):<13} {str(row[3]):<14} "
                    f"{str(row[4]):<10} {str(row[5]):<7} {str(row[6]):<8} {row[7]}\n")

def save_csv(path):
    with open(path, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(headers)
        writer.writerows(table_rows)

def save_html(path):
    with open(path, 'w', encoding='utf-8') as f:
        f.write(f"<h2>Record #{record_number}</h2>\n<table border='1'>\n<tr>")
        f.write("".join(f"<th>{h}</th>" for h in headers) + "</tr>\n")
        for row in table_rows:
            f.write("<tr>" + "".join(f"<td>{str(cell)}</td>" for cell in row) + "</tr>\n")
        f.write("</table>\n")

def save_md(path):
    with open(path, 'w', encoding='utf-8') as f:
        f.write(f"# Record #{record_number}\n\n")
        f.write("| " + " | ".join(headers) + " |\n")
        f.write("|" + "|".join(["-" * len(h) for h in headers]) + "|\n")
        for row in table_rows:
            f.write("| " + " | ".join(str(cell) for cell in row) + " |\n")

def save_xlsx(path):
    if not openpyxl:
        print("⚠️ Cannot save to Excel: openpyxl is not installed.")
        return
    wb = Workbook()
    ws = wb.active
    ws.title = f"Record_{record_number}"
    ws.append(headers)
    for row in table_rows:
        ws.append(row)
    wb.save(path)

# Save outputs
if "txt" in output_formats:
    save_txt(f"{base_filename}.txt")
if "csv" in output_formats:
    save_csv(f"{base_filename}.csv")
if "html" in output_formats:
    save_html(f"{base_filename}.html")
if "md" in output_formats:
    save_md(f"{base_filename}.md")
if "xlsx" in output_formats:
    save_xlsx(f"{base_filename}.xlsx")

print("\n✅ Export complete.")
```

---

### 🧪 Para probarlo:

1. Ejecutá el script sin tener un archivo `.env`:

   ```bash
   python script.py 1
   ```

2. Verás que crea `.env` y te pide completarlo.

3. Una vez completado, podés correrlo de nuevo con formatos opcionales:

   ```bash
   python script.py 1 --formats txt md html
   ```

---

¿Querés que empaquete esto como un proyecto con README y dependencias (`requirements.txt`)?
